## 주유소 정보 페이지 띄우기

In [94]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

url = "https://www.opinet.co.kr/searRgSelect.do"
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.set_window_size(1400, 1000)

driver.get(url)

## 서울시 내 구 데이터부터 찾기

In [95]:
from selenium.webdriver.common.by import By

sido = driver.find_element(By.ID, "SIDO_NM0")
sido_list = sido.find_elements(By.TAG_NAME, "option")
for idx, sido_name in enumerate(sido_list):
    print(str(idx) + ". " + sido_name.get_attribute("value"))

0. 
1. 서울특별시
2. 부산광역시
3. 대구광역시
4. 인천광역시
5. 광주광역시
6. 대전광역시
7. 울산광역시
8. 세종특별자치시
9. 경기도
10. 강원특별자치도
11. 충청북도
12. 충청남도
13. 전북특별자치도
14. 전라남도
15. 경상북도
16. 경상남도
17. 제주특별자치도


In [96]:
sido_names = [sido_name.get_attribute("value") for sido_name in sido_list]
sido_names

['',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [97]:
sido_names = sido_names[1:]
sido_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [98]:
print(sido_names[0])
sido.send_keys(sido_names[0])

서울특별시


In [99]:
gu = driver.find_element(By.ID, "SIGUNGU_NM0")
gu_list = gu.find_elements(By.TAG_NAME, "option")

gu_names = [gu_name.get_attribute("value") for gu_name in gu_list]
print(len(gu_names), gu_names)

26 ['', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


In [100]:
gu_names.remove('')
print(len(gu_names), gu_names)

25 ['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']


## 특정 구의 주유소 팝업창 띄우기

In [43]:
gu.send_keys(gu_names[12])

In [44]:
oil_list = driver.find_element(By.ID, "body1")
station = oil_list.find_elements(By.TAG_NAME, "a")

station_temp = station[2]
station_temp.click()

## 팝업창에서 정보 추출

In [45]:
popup = driver.find_element(By.CLASS_NAME, "atlanmap-info-content")

name = popup.find_element(By.ID, "os_nm") # 주유소 이름
phone_num = popup.find_element(By.ID, "phn_no") # 주유소 전화번호
addr = popup.find_element(By.ID, "rd_addr") # 주유소 주소
brand = popup.find_element(By.ID, "poll_div_nm") # 주유소 상표

# 유가정보
price_table = popup.find_element(By.ID, "infoTbody")
price_cont = price_table.find_elements(By.TAG_NAME, "tr")
price_info = {}
for line in price_cont:
    oil_type = line.find_elements(By.TAG_NAME, "td")[0].text # 유종
    oil_price = line.find_elements(By.TAG_NAME, "td")[1].find_element(By.TAG_NAME, "label").text # 가격

    price_info[oil_type] = oil_price

# 부가정보
service = popup.find_element(By.CLASS_NAME, "service")
service_img = service.find_elements(By.TAG_NAME, "img")
service_list = []
for img in service_img:
    # image 이름 기준으로 부가 서비스 유무 파악
    if "off" not in img.get_attribute("src"):
        service_list.append(img.get_attribute("alt"))

# 각 정보 출력
print(name.text)
print(phone_num.text)
print(addr.text)
print(brand.text)

for key, value in price_info.items():
    print(key, value)

print(service_list)

청원주유소
02-325-5533
서울 마포구 월드컵북로 113 (성산동)
HD현대오일뱅크
고급휘발유 1,978
보통휘발유 1,659
경유 1,569
 
['세차장']


## 특정 구 안에서 정보 추출 테스트

In [57]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

oil_list = driver.find_element(By.ID, "body1")
stations = oil_list.find_elements(By.TAG_NAME, "a")

name_list = []
phn_list = []
addr_list = []
brand_list = []
price_list = []
service_list = []

for station in stations:
    station.click()
    time.sleep(0.5)

    popup = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.CLASS_NAME, "atlanmap-info-content")))

    name = popup.find_element(By.ID, "os_nm") # 주유소 이름
    phone_num = popup.find_element(By.ID, "phn_no") # 주유소 전화번호
    addr = popup.find_element(By.ID, "rd_addr") # 주유소 주소
    brand = popup.find_element(By.ID, "poll_div_nm") # 주유소 상표

    # 유가정보
    price_table = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, "infoTbody")))
    price_cont = price_table.find_elements(By.TAG_NAME, "tr")
    price_info = {}
    for line in price_cont:
        oil_type = line.find_elements(By.TAG_NAME, "td")[0].text # 유종
        oil_price = line.find_elements(By.TAG_NAME, "td")[1].find_element(By.TAG_NAME, "label").text # 가격

        price_info[oil_type] = oil_price

    # 부가정보
    service = popup.find_element(By.CLASS_NAME, "service")
    service_img = service.find_elements(By.TAG_NAME, "img")
    services = []
    for img in service_img:
        # image 이름 기준으로 부가 서비스 유무 파악
        if "off" not in img.get_attribute("src"):
            services.append(img.get_attribute("alt"))

    name_list.append(name.text)
    phn_list.append(phone_num.text)
    addr_list.append(addr.text)
    brand_list.append(brand.text)
    price_list.append(price_info)
    service_list.append(services)


In [58]:
# 리스트 길이 확인
len(name_list), len(phn_list), len(addr_list), len(brand_list), len(price_list), len(service_list)

(12, 12, 12, 12, 12, 12)

In [59]:
print(name_list[0])

마포시엠주유소


## Dataframe 변환 연습(특정 구만)

In [60]:
# dataframe으로 변환
import pandas as pd

# 리스트를 문자열로 변환하는 함수
def clean_list_to_string(x):
    if isinstance(x, list):
        return ", ".join(x)  # 리스트의 요소들을 쉼표로 구분하여 문자열로 변환
    return x

oil_data = {'상호':name_list, 
            '전화번호':phn_list, 
            "주소":addr_list,
            "상표":brand_list,
            "유종별 가격":price_list,
            "부가정보":service_list}

oil_df = pd.DataFrame(oil_data)

oil_df.sort_index(inplace=True)
oil_df['부가정보'] = oil_df['부가정보'].apply(clean_list_to_string)

oil_df

,상호,전화번호,주소,상표,유종별 가격,부가정보
0,마포시엠주유소,02-338-9447,서울 마포구 월드컵북로 62,SK에너지,"{'': '', '보통휘발유': '1,659', '경유': '1,539'}",
1,청원주유소,02-325-5533,서울 마포구 월드컵북로 113 (성산동),HD현대오일뱅크,"{'고급휘발유': '1,978', '보통휘발유': '1,659', '경유': '1,...",세차장
2,(주)타이거통상 월드컵주유소,070-4121-0021,서울 마포구 월드컵북로 170 (성산제2동),S-OIL,"{'고급휘발유': '1,889', '보통휘발유': '1,659', '경유': '1,...",
3,성산대교셀프주유소,02-323-0078,서울 마포구 성산로 144,HD현대오일뱅크,"{'': '', '보통휘발유': '1,659', '경유': '1,569', '실내등...","세차장, 경정비"
4,(주)안국에너지 망원동주유소,02-334-0917,서울 마포구 월드컵로 119 (망원동),SK에너지,"{'': '', '보통휘발유': '1,758', '경유': '1,628'}",세차장
5,에쓰오일(주) 염리동주유소,02-3272-5155,서울 마포구 백범로 126 (염리동),S-OIL,"{'': '', '보통휘발유': '1,779', '경유': '1,669'}",세차장
6,용강주유소,070-7795-4800,서울 마포구 토정로 258,S-OIL,"{'': '', '보통휘발유': '1,779', '경유': '1,669', '실내등...",
7,마포주유소,02-711-5189,서울 마포구 마포대로 186,S-OIL,"{'고급휘발유': '1,942', '보통휘발유': '1,792', '경유': '1,...",세차장
8,에스에스오토셀프,02-336-5185,서울 마포구 서강로 76 (창전동),HD현대오일뱅크,"{'고급휘발유': '2,098', '보통휘발유': '1,799', '경유': '1,...","세차장, 24시영업"
9,(주)만성상사 대흥주유소,02-3273-5151,서울 마포구 대흥로 61 (대흥동),SK에너지,"{'고급휘발유': '2,199', '보통휘발유': '1,799', '경유': '1,...","세차장, 24시영업"


In [61]:
# 유종별 가격 열 변환
oil_df = pd.concat([oil_df.drop(columns=['유종별 가격']), oil_df['유종별 가격'].apply(pd.Series)], axis=1)
oil_df

,상호,전화번호,주소,상표,부가정보,,보통휘발유,경유,고급휘발유,실내등유
0,마포시엠주유소,02-338-9447,서울 마포구 월드컵북로 62,SK에너지,,,"1,659","1,539",NaN,NaN
1,청원주유소,02-325-5533,서울 마포구 월드컵북로 113 (성산동),HD현대오일뱅크,세차장,,"1,659","1,569","1,978",NaN
2,(주)타이거통상 월드컵주유소,070-4121-0021,서울 마포구 월드컵북로 170 (성산제2동),S-OIL,,,"1,659","1,539","1,889",NaN
3,성산대교셀프주유소,02-323-0078,서울 마포구 성산로 144,HD현대오일뱅크,"세차장, 경정비",,"1,659","1,569",NaN,"1,649"
4,(주)안국에너지 망원동주유소,02-334-0917,서울 마포구 월드컵로 119 (망원동),SK에너지,세차장,,"1,758","1,628",NaN,NaN
5,에쓰오일(주) 염리동주유소,02-3272-5155,서울 마포구 백범로 126 (염리동),S-OIL,세차장,,"1,779","1,669",NaN,NaN
6,용강주유소,070-7795-4800,서울 마포구 토정로 258,S-OIL,,,"1,779","1,669",NaN,"1,650"
7,마포주유소,02-711-5189,서울 마포구 마포대로 186,S-OIL,세차장,,"1,792","1,682","1,942",NaN
8,에스에스오토셀프,02-336-5185,서울 마포구 서강로 76 (창전동),HD현대오일뱅크,"세차장, 24시영업",,"1,799","1,689","2,098",NaN
9,(주)만성상사 대흥주유소,02-3273-5151,서울 마포구 대흥로 61 (대흥동),SK에너지,"세차장, 24시영업",,"1,799","1,689","2,199","1,745"


## 서울 시내 전체에 적용해보기

In [101]:
from tqdm.notebook import tqdm

def get_station_info(gu_names):
    name_list = []
    phn_list = []
    addr_list = []
    brand_list = []
    price_list = []
    service_list = []
    
    for idx, gu_name in tqdm(enumerate(gu_names), total=len(gu_names)):
        gu = driver.find_element(By.ID, "SIGUNGU_NM0")
        gu.send_keys(gu_name)
        time.sleep(3)
        driver.refresh()

        oil_list = driver.find_element(By.ID, "body1")
        stations = oil_list.find_elements(By.TAG_NAME, "a")

        for station in tqdm(stations):
            station.click()
            time.sleep(0.5)

            popup = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "atlanmap-info-content")))

            name = popup.find_element(By.ID, "os_nm") # 주유소 이름
            phone_num = popup.find_element(By.ID, "phn_no") # 주유소 전화번호
            addr = popup.find_element(By.ID, "rd_addr") # 주유소 주소
            brand = popup.find_element(By.ID, "poll_div_nm") # 주유소 상표

            # 유가정보
            price_table = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, "infoTbody")))
            price_cont = price_table.find_elements(By.TAG_NAME, "tr")
            price_info = {}
            for line in price_cont:
                oil_type = line.find_elements(By.TAG_NAME, "td")[0] # 유종
                oil_price = line.find_elements(
                    By.TAG_NAME, "td")[1].find_element(By.TAG_NAME, "label") # 가격

            price_info[oil_type.text] = oil_price.text

            # 부가정보
            service = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "service")))
            service_img = service.find_elements(By.TAG_NAME, "img")
            services = []
            for img in service_img:
                # image 이름 기준으로 부가 서비스 유무 파악
                if "off" not in img.get_attribute("src"):
                    services.append(img.get_attribute("alt"))

            name_list.append(name.text)
            phn_list.append(phone_num.text)
            addr_list.append(addr.text)
            brand_list.append(brand.text)
            price_list.append(price_info)
            service_list.append(services)

    return name_list, phn_list, addr_list, brand_list, price_list, service_list

In [102]:
total_name, total_phn, total_addr, total_brand, total_price, total_service = get_station_info(gu_names)
# 리스트 길이 확인
len(total_name), len(total_phn), len(total_addr), len(total_brand), len(total_price), len(total_service)


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

(422, 422, 422, 422, 422, 422)

In [119]:
print(total_price[0])

{'': ''}


## 전체 데이터 Dataframe으로 변환 및 정보 수정

In [113]:
seoul_oil_data = {'상호':total_name, 
            '전화번호':total_phn, 
            "주소":total_addr,
            "상표":total_brand,
            "유종별 가격":total_price,
            "부가정보":total_service}

seoul_oil_df = pd.DataFrame(seoul_oil_data)

seoul_oil_df.sort_index(inplace=True)
seoul_oil_df['부가정보'] = seoul_oil_df['부가정보'].apply(clean_list_to_string) # 문자열 형태로 표시

# 유종별 가격 열 변환
seoul_oil_df = pd.concat(
    [seoul_oil_df.drop(columns=['유종별 가격']), seoul_oil_df['유종별 가격'].apply(pd.Series)], axis=1)

seoul_oil_df.tail(20)

,상호,전화번호,주소,상표,부가정보,
402,장충드림주유소,02-2275-5150,서울 중구 동호로 296,S-OIL,세차장,
403,약수주유소,02-2252-3345,서울 중구 다산로 173,SK에너지,"세차장, 경정비, 24시영업",
404,HD현대오일뱅크㈜직영 장원주유소,02-2236-2759,서울 중구 동호로 203 (신당동),HD현대오일뱅크,,
405,SK에너지(주) 퇴계로주유소,02-2273-5189,서울 중구 퇴계로 228 (필동2가),SK에너지,,
406,장충주유소,02-2279-9965,서울 중구 장충단로 202 (장충동1가),SK에너지,"세차장, 24시영업",
407,신당동주유소,02-2252-6688,서울 중구 다산로 242 (신당동),SK에너지,"세차장, 24시영업",
408,필동주유소,02-2267-8025,서울 중구 퇴계로 196,GS칼텍스,24시영업,
409,서남주유소,02-752-9475,서울 중구 통일로 30,SK에너지,"세차장, 24시영업",
410,우림주유소,02-433-9990,서울 중랑구 용마산로 487 (망우제3동),GS칼텍스,세차장,
411,(주)자연에너지 대창주유소,02-434-1448,서울 중랑구 동일로 636 (면목동),HD현대오일뱅크,,


In [114]:
seoul_oil_df["구"] = [address.split()[1] for address in seoul_oil_df["주소"]]
seoul_oil_df.head()

,상호,전화번호,주소,상표,부가정보,,구
0,지에스칼텍스㈜ 은마주유소,02-3452-1882,서울 강남구 영동대로 235,GS칼텍스,"세차장, 경정비",,강남구
1,HD현대오일뱅크㈜직영 강남셀프주유소,02-3462-0018,서울 강남구 도곡로 208,HD현대오일뱅크,세차장,,강남구
2,(주)보성 세곡주유소,02-445-6870,서울 강남구 헌릉로 731 (세곡동),SK에너지,세차장,,강남구
3,방죽주유소,02-459-3434,서울 강남구 밤고개로 215 (율현동),GS칼텍스,세차장,,강남구
4,HD현대오일뱅크㈜직영 도곡셀프주유소,02-529-5101,서울 강남구 남부순환로 2718,HD현대오일뱅크,"세차장, 경정비",,강남구


In [115]:
seoul_oil_df["구"].unique(), len(seoul_oil_df["구"].unique())

(array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
        '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
        '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object),
 25)

In [116]:
seoul_oil_df.to_csv("../data/seoul_station_info.csv", sep=",", encoding="utf-8")

In [117]:
driver.quit()